In [ ]:
import json
import pandas as pd
from itertools import combinations

# CSV 파일 읽기
file_path = "Personality_Traits_Dataset-40.csv"
df = pd.read_csv(file_path)

# 컬럼 이름 지정
df.columns = ["이름", "성실성", "친화성", "개방성", "외향성", "신경증"]

# 가중치 설정
weights = {
    "성실성 평균": 20.22,
    "친화성 평균": 20.22,
    "성실성 유사도": 16.18,
    "친화성 유사도": 16.18,
    "개방성 다양성": 12.14,
    "외향성 다양성": 8.09,
    "신경증 유사도": 4.05,
    "개방성 평균": 0.97,
    "외향성 평균": 0.97,
    "신경증 평균": 0.97,
}

threshold_score = 55  # 팀 매칭 성립 기준 점수

# 글로벌 통계 계산
global_std = df[["성실성", "친화성", "신경증"]].std().to_dict()
global_mean = df[["성실성", "친화성", "개방성", "외향성", "신경증"]].mean().to_dict()
global_variance = df[["개방성", "외향성"]].var().to_dict()

# 점수 계산 함수
def calculate_team_score(team_df):
    score = 0

    # 유사도 평가
    for feature in ["성실성", "친화성", "신경증"]:
        std_value = team_df[feature].std()
        max_std = global_std[feature]
        weight = weights[f"{feature} 유사도"]

        # 유사성 점수 계산
        similarity_score = max(0, 1 - (std_value / max_std))
        score += similarity_score * weight

    # 평균 평가
    for feature in ["성실성", "친화성", "개방성", "외향성", "신경증"]:
        mean_value = team_df[feature].mean()
        target_mean = global_mean[feature]
        weight = weights.get(f"{feature} 평균", 0)

        # 평균 점수 계산
        mean_score = max(0, 1 - (abs(mean_value - target_mean) / target_mean))
        score += mean_score * weight

    # 다양성 평가
    for feature in ["개방성", "외향성"]:
        variance = team_df[feature].var()
        max_variance = global_variance[feature]
        weight = weights[f"{feature} 다양성"]

        # 다양성 점수 계산
        diversity_score = max(0, 1 - (variance / max_variance))
        score += diversity_score * weight

    return round(score, 2)

# 팀 구성 함수
def form_best_teams(df, team_size=4, threshold_score=threshold_score, max_teams=20):
    selected_teams = []
    used_indices = set()

    while len(selected_teams) < max_teams and len(df) - len(used_indices) >= team_size:
        available_indices = [i for i in df.index if i not in used_indices]
        all_combinations = combinations(available_indices, team_size)

        best_team = None
        best_score = -1

        for combination in all_combinations:
            team_df = df.loc[list(combination)]
            team_score = calculate_team_score(team_df)

            if team_score > best_score and team_score >= threshold_score:
                best_team = (team_df, team_score)
                best_score = team_score

        if best_team:
            selected_teams.append(best_team)
            used_indices.update(best_team[0].index)
        else:
            break

    return selected_teams

# 팀 구성 실행
team_size = 4
teams = form_best_teams(df, team_size)

# 결과 출력
for i, (team_df, score) in enumerate(teams):
    print(f"Team {i + 1}:\n")
    for _, row in team_df.iterrows():
        print(f"[{row['이름']}, {row['성실성']}, {row['친화성']}, {row['개방성']}, {row['외향성']}, {row['신경증']}]")
    print(f"Team Score: {score}")
    print("\n" + "=" * 50 + "\n")

# JSON 형식으로 저장할 데이터 생성
team_data = {}
for i, (team_df, score) in enumerate(teams, 1):  # (team_df, score)로 언패킹
    team_name = f"Team {i}"
    team_data[team_name] = [
        {
            "이름": row["이름"],
            "성실성": int(row["성실성"]),
            "친화성": int(row["친화성"]),
            "개방성": int(row["개방성"]),
            "외향성": int(row["외향성"]),
            "신경증": int(row["신경증"]),
        }
        for _, row in team_df.iterrows()
    ]

# JSON 파일로 저장
json_file_path = "final_teams.json"
with open(json_file_path, "w", encoding="utf-8") as json_file:
    json.dump(team_data, json_file, ensure_ascii=False, indent=4)

Team 1:

[Han Eunwoo, 84, 68, 43, 34, 80]
[Kim Sia, 91, 67, 39, 31, 78]
[Jo Jihun, 82, 71, 38, 32, 74]
[Jung Taeyoon, 83, 69, 36, 50, 89]
Team Score: 84.84


Team 2:

[Kang Seojin, 75, 43, 62, 72, 81]
[Choi Woojin, 76, 55, 70, 73, 64]
[Kang Juwon, 77, 95, 60, 72, 63]
[Seo Woojin, 75, 84, 62, 77, 86]
Team Score: 76.34


Team 3:

[Choi Jiah, 64, 67, 52, 70, 74]
[Jung Seojun, 64, 60, 66, 66, 47]
[Jo Hayoon, 62, 76, 78, 58, 65]
[Kim Yejoon, 66, 79, 59, 88, 87]
Team Score: 74.59


Team 4:

[Yoo Sieun, 55, 74, 38, 40, 51]
[Han Doyoung, 59, 51, 30, 40, 66]
[Park Seojun, 45, 64, 40, 59, 65]
[Choi Minji, 44, 62, 39, 42, 64]
Team Score: 70.2


Team 5:

[Park Doyoon, 44, 65, 74, 61, 84]
[Park Daeun, 54, 81, 80, 65, 62]
[Seo Minjae, 45, 79, 89, 74, 79]
[Yoon Seoa, 43, 69, 72, 37, 71]
Team Score: 69.02


Team 6:

[Hong Sua, 80, 69, 33, 84, 42]
[Yoon Gunwoo, 45, 68, 32, 81, 78]
[Choi Yena, 52, 78, 41, 74, 33]
[Jung Hayul, 80, 81, 41, 89, 56]
Team Score: 67.89


Team 7:

[Jang Yujin, 82, 93, 84, 90, 

In [ ]:
import json
import pandas as pd

# JSON 파일 경로
json_file_path = "final_teams.json"

# JSON 파일 로드 함수
def load_teams_from_json(file_path):
    with open(file_path, "r", encoding="utf-8") as json_file:
        return json.load(json_file)

# JSON 파일에서 데이터를 로드하여 teams에 할당
teams = load_teams_from_json(json_file_path)

# 유사성을 기준으로 할 특성
similarity_features = ['성실성', '친화성', '신경증']

# 다양성을 기준으로 할 특성
diversity_features = ['개방성', '외향성']

# 평가 점수를 기준으로 범위를 판별하는 함수
def evaluate_score(mean, std, value):
    high_threshold = mean + std
    low_threshold = mean - std

    if value > high_threshold:
        return "높다"
    elif mean < value <= high_threshold:
        return "다소 높다"
    elif low_threshold <= value < mean:
        return "다소 낮다"
    else:
        return "낮다"

# 팀 평가 함수
def evaluate_team(team_df, global_means, global_stds):
    evaluation = {}

    # 유사성 평가
    for feature in similarity_features:
        mean_value = round(team_df[feature].mean(), 2)
        std_value = round(team_df[feature].std(), 2)
        global_mean = global_means[feature]
        global_std = global_stds[feature]

        # 유사도 평가
        cv = round((std_value / mean_value) * 100, 2) if mean_value != 0 else float('inf')
        evaluation[f"{feature} 유사도"] = min(cv, 50)
        evaluation[f"{feature} 유사도 평가"] = (
            "높다" if cv <= 15 else
            "평균적인 수준이다" if cv <= 20 else
            "낮다"
        )

        # 평균 평가
        evaluation[f"{feature} 평균"] = mean_value
        evaluation[f"{feature} 평균 평가"] = evaluate_score(global_mean, global_std, mean_value)

    # 다양성 평가
    for feature in diversity_features:
        scores = team_df[feature].values
        global_mean = global_means[feature]
        global_std = global_stds[feature]

        diversity_levels = [
            "높다" if score > global_mean + global_std else
            "낮다" if score < global_mean - global_std else
            "평균적인 수준이다"
            for score in scores
        ]
        unique_levels = len(set(diversity_levels))
        evaluation[f"{feature} 다양성 평가"] = (
            "높다" if unique_levels >= 4 else
            "다소 높다" if unique_levels == 3 else
            "다소 낮다" if unique_levels == 2 else
            "낮다"
        )

    return evaluation

# 글로벌 평균 및 표준편차 계산
all_data = []
for team in teams.values():
    all_data.extend(team)
df = pd.DataFrame(all_data)

# 숫자형 데이터만 선택하여 글로벌 메트릭 계산
numeric_df = df.select_dtypes(include=['number'])
global_means = numeric_df.mean()
global_stds = numeric_df.std()

# 평가 결과에 따른 설명 추가
def add_evaluation_description(key, value):
    descriptions = {
        "성실성 유사도 평가": {
            "높다": "업무 우선순위와 작업 속도에 대한 기대치가 유사하여 의사소통 과정에서 충돌이 적을 것으로 기대됩니다.\n팀원 간 기대치가 비슷하기 때문에 스트레스를 유발할 수 있는 오해가 줄어들고 협업 시 마찰이 적을 가능성이 높습니다.",
            "다소 높다": "대체로 비슷한 성실성을 지닌 팀원들로 구성되어 서로의 행동 방식을 더 잘 이해하고 신뢰하여 협력이 원활하게 이루어질 가능성이 높습니다.",
        },
        "친화성 유사도 평가": {
            "높다": "팀원들의 대인 관계 방식이 유사하므로 상호작용에서 예상치 못한 마찰이 적을 것으로 기대됩니다.\n업무 진행 방식이나 목표 설정에서도 모두가 같은 방향으로 나아가기 쉬워 생산성이 높을 것으로 기대됩니다.",
            "다소 높다": "대체로 비슷한 친화성을 지닌 팀원들로 협동이 중요한 환경에서 장점을 가질 것으로 예상됩니다.",
        },
        "신경증 유사도 평가": {
            "높다": "팀원들의 감정적 반응과 스트레스 대처 방식이 유사하여 서로를 잘 이해하고 배려하며 협력할 것으로 기대됩니다.",
            "다소 높다": "팀원들의 정서적 안정감이 대체로 비슷한 수준을 보여 긴장이나 스트레스 상황에서도 안정적인 협력을 할 가능성이 높습니다.",
        },
        "성실성 평균 평가": {
            "높다": "팀의 성실성 평균이 높아 안정적이고 효율적인 진행이 가능하며 지속적으로 목표를 향해 나아가 외부에서 신뢰받는 팀으로 평가받을 가능성이 높습니다.",
            "다소 높다": "팀의 성실성 평균이 다소 높아 공동 목표를 달성하기 위해 각자의 역할을 충실히 수행하여 프로젝트 수행과정이 안정적일 것으로 기대됩니다.",
        },
        "친화성 평균 평가": {
            "높다": "팀의 친화성 평균이 높아 대립보다는 화합을 중시하여 스트레스가 적고 협력적인 분위기가 유지될 것으로 기대됩니다.\n서로를 신뢰하고 돕는 태도가 강해 팀워크가 자연스럽게 강화될 것입니다.",
            "다소 높다": "친화성 평균이 다소 높아 대체로 원활한 소통과 협력이 가능하며 안정적인 팀 분위기가 예상됩니다.",
        },
        "개방성 다양성 평가": {
            "높다": "팀 내 다양한 시각이 공존하여 창의적인 아이디어와 실질적이고 현실적인 실행 계획으로 혁신적인 결과를 도출할 가능성이 큽니다.",
            "다소 높다": "다양한 관점을 통해 기존 방식과 새로운 접근법의 균형을 맞출 수 있을 것으로 기대됩니다.",
            "다소 낮다": "창의성, 호기심, 새로운 경험을 추구하려는 성향이 비슷해 가치관과 일하는 방식이 비슷하여 충돌이 적을 것으로 기대됩니다.",
            "낮다": "대부분의 팀원이 비슷한 사고방식을 공유하며, 합의에 이르는 데 시간이 적게 걸리고 자연스럽게 같은 방향으로 나아갈 가능성이 높습니다.",
        },
        "외향성 다양성 평가": {
            "높다": "외향적인 사람은 팀의 분위기를 주도하고 활발한 논의를 이끌어내며 내향적인 사람은 더 깊이 있는 사고와 신중한 발언으로 논의를 보완합니다.\n서로의 강점을 활용하여 팀의 성과를 극대화할 것으로 기대됩니다.",
            "다소 높다": "팀 내 에너지가 한쪽으로 치우치지 않고 지속적으로 유지되며 의견 표현과 경청의 균형이 이루어져 더 풍부한 논의가 가능할 것으로 기대됩니다.",
            "다소 낮다": "외향성에서 약간의 차이는 있으나, 대체로 비슷한 방식으로 소통하며 협력할 가능성이 있습니다.",
            "낮다": "팀의 성향에 따라 장점이 뚜렷해지며, 특정한 환경이나 과제에 더욱 효과적일 것으로 예상됩니다.",
        },
    }
    return descriptions.get(key, {}).get(value, "")

# 팀원 정보, 팀 평가, 설명 출력
def evaluate_all_teams(teams, global_means, global_stds):
    evaluations = {}
    for team_name, team_data in teams.items():
        print(f"\n{'=' * 50}\nTeam: {team_name}")

        # 팀원 정보 출력
        print("\nTeam Members:")
        for member in team_data:
            print(f"  Name: {member['이름']}, 성실성: {member['성실성']}, 친화성: {member['친화성']}, "
                  f"개방성: {member['개방성']}, 외향성: {member['외향성']}, 신경증: {member['신경증']}")

        team_df = pd.DataFrame(team_data)
        evaluations[team_name] = evaluate_team(team_df, global_means, global_stds)

        # 팀 평가 결과 출력
        print("\nEvaluation Results:")
        full_descriptions = []
        for key, value in evaluations[team_name].items():
            print(f"  {key}: {value}")
            description = add_evaluation_description(key, value)
            if description:
                full_descriptions.append(f"  {key}: {description}")

        # 평가에 대한 설명 출력
        if full_descriptions:
            print("\nDetailed Explanations:")
            print("\n".join(full_descriptions))
    return evaluations

# 평가 실행
team_evaluations = evaluate_all_teams(teams, global_means, global_stds)




Team: Team 1

Team Members:
  Name: Han Eunwoo, 성실성: 84, 친화성: 68, 개방성: 43, 외향성: 34, 신경증: 80
  Name: Kim Sia, 성실성: 91, 친화성: 67, 개방성: 39, 외향성: 31, 신경증: 78
  Name: Jo Jihun, 성실성: 82, 친화성: 71, 개방성: 38, 외향성: 32, 신경증: 74
  Name: Jung Taeyoon, 성실성: 83, 친화성: 69, 개방성: 36, 외향성: 50, 신경증: 89

Evaluation Results:
  성실성 유사도: 4.8
  성실성 유사도 평가: 높다
  성실성 평균: 85.0
  성실성 평균 평가: 다소 높다
  친화성 유사도: 2.49
  친화성 유사도 평가: 높다
  친화성 평균: 68.75
  친화성 평균 평가: 다소 낮다
  신경증 유사도: 7.9
  신경증 유사도 평가: 높다
  신경증 평균: 80.25
  신경증 평균 평가: 다소 높다
  개방성 다양성 평가: 다소 낮다
  외향성 다양성 평가: 다소 낮다

Detailed Explanations:
  성실성 유사도 평가: 업무 우선순위와 작업 속도에 대한 기대치가 유사하여 의사소통 과정에서 충돌이 적을 것으로 기대됩니다.
팀원 간 기대치가 비슷하기 때문에 스트레스를 유발할 수 있는 오해가 줄어들고 협업 시 마찰이 적을 가능성이 높습니다.
  성실성 평균 평가: 팀의 성실성 평균이 다소 높아 공동 목표를 달성하기 위해 각자의 역할을 충실히 수행하여 프로젝트 수행과정이 안정적일 것으로 기대됩니다.
  친화성 유사도 평가: 팀원들의 대인 관계 방식이 유사하므로 상호작용에서 예상치 못한 마찰이 적을 것으로 기대됩니다.
업무 진행 방식이나 목표 설정에서도 모두가 같은 방향으로 나아가기 쉬워 생산성이 높을 것으로 기대됩니다.
  신경증 유사도 평가: 팀원들의 감정적 반응과 스트레스 대처 방식이 유사하여 서로를 잘 이해하고 배려하며 협력할 것으로 기대됩니다.
  개방성